# Atelier Dataviz & Javascript

Cet atelier est la suite du cours dont le thème est la visualisation de données avec les librairies JavaScript de Python et R.
JavaScript permet d'apporter des fonctionnalités dynamiques (interactives) aux visualisations de données.
Ici nous allons travailler avec des données connues, celles de Lig'Air, et utiliser 3 fonctionnalités JavaScript :
- l'édition dynamique de table de résultats
- l'affichage de courbes interactives
- l'affichage de heatmap interactives (courbes de chaleur)

## Chargement des librairies

In [9]:
import requests
from pandas.io.json import json_normalize
import pandas as pd

## Importation des données

Les données importées ici sont les indices ATMO par jour et par ville, disponibles sur le portail open data de Lig'Air.

In [10]:
# Url des données de l'API Lig'Air
myUrl = "https://services1.arcgis.com/HzzPcgRsxxyIZdlU/arcgis/rest/services/Indices_de_qualit%C3%A9_de_l_air/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"
request = requests.get(url=myUrl).json()

# Extraction des données et transposition en dataFrame...
result = json_normalize(request['features'])
df = pd.DataFrame(result)

# Conversion du champ 'attributes.date_echeance' en type 'date'
df['attributes.date_echeance'] = pd.to_datetime(df['attributes.date_echeance'])

## Visualisation de la table

La librairie 'qgrid' est ici utilisée, conçue pour Jupyter Notebook (il est nécessaire de l'installer si ce n'est pas déjà fait).

In [3]:
import qgrid
edit_df = qgrid.show_grid(df, show_toolbar=True)
edit_df

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Visualisation de courbes avec Highcharts.js

Avant de pouvoir visualiser convenablement nos courbes d'indice ATMO, il est nécessaire de transposer les données au format large (format attendu pour Highcharts et Heatmap), avec la fonction pivot de pandas qui nécessite 3 arguments :
- index = date
- colonne = villes
- remplissage avec les valeurs ATMO

In [4]:
df_large = df.pivot(index='attributes.date_echeance', columns='attributes.nom_zone', values='attributes.valeur')
df_large.head

<bound method NDFrame.head of attributes.nom_zone       Blois  Bourges  Chartres  Chateauroux  Dreux  \
attributes.date_echeance                                                 
2018-10-07                  NaN      NaN       NaN          NaN    3.0   
2018-10-08                  3.0      4.0       4.0          3.0    4.0   
2018-10-09                  4.0      4.0       4.0          4.0    4.0   
2018-10-10                  4.0      4.0       4.0          4.0    4.0   
2018-10-11                  4.0      4.0       3.0          4.0    3.0   
2018-10-12                  3.0      3.0       3.0          4.0    3.0   
2018-10-13                  4.0      4.0       3.0          4.0    3.0   
2018-10-14                  4.0      4.0       3.0          4.0    3.0   
2018-10-15                  4.0      4.0       4.0          4.0    4.0   
2018-10-16                  4.0      4.0       4.0          4.0    4.0   
2018-10-17                  4.0      5.0       3.0          4.0    4.0   
2018-10-

In [5]:
#il est nécessaire d'installer 'highchart' si ce n'est pas déjà fait
from highcharts import Highchart # import highchart library
H = Highchart() # setup highchart instance
for i in df_large.columns:
    H.add_data_set(list(df_large[i]), 'spline', i)
H

In [6]:
# Ajout d'informations au graphique, considérées comme des options par highcharts
options = {
    'title': {
        'text': 'Indice ATMO dans les principales villes de la région Centre-Val de Loire'
    },
    'xAxis': { 
        'title': {
            'text': 'Date (#affichage à finaliser)'
        }
    },
    'yAxis': { 
        'title': {
            'text': 'Indice ATMO'
        }
    }
}
H.set_dict_options(options)
H

## Dataviz - Heatmap

### Transposition des données 'indice ATMO' sous forme de matrice

In [7]:
#il est nécessaire d'installer 'plotly' si ce n'est pas déjà fait
# Connexion à l'API avec un compte existant
import plotly
plotly.tools.set_credentials_file(username='RemiJulien', api_key='P7zRkAFgDxfcFdcGwzTP')

# Chargement des librairies
import plotly.plotly as py
import plotly.graph_objs as go

# Création de la 'heatmap'
heatmap = go.Heatmap(z=df_large.as_matrix(), x=df_large.columns, y=df_large.index, colorscale='Jet')
data = [heatmap]
py.iplot(data, filename='ATMO-heatmap')

/home/moi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/home/moi/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



Afin d'améliorer le rendu visuel de la 'heatmap', il peut être judicieux de transposer la matrice de données, avec la fonction '.T'

In [8]:
heatmap = go.Heatmap(z=df_large.as_matrix().T, x=df_large.index, y=df_large.columns, colorscale='Jet')
data = [heatmap]
py.iplot(data, filename='ATMO-heatmap')

/home/moi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

